<a href="https://colab.research.google.com/github/marcellosusanto11/predict_ncaa_tournament/blob/master/Scraper/scrape_teamranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
# Set working directory
%cd drive/My\ Drive/Colab\ Notebooks/Tugas\ Akhir/

/content/drive/My Drive/Colab Notebooks/Tugas Akhir


In [0]:
# Install package


In [0]:
# Setting general package
import pandas as pd
import pandas_profiling as pp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
%matplotlib inline

from matplotlib.pylab import rcParams
# For every plotting cell use this
# grid = gridspec.GridSpec(n_row,n_col)
# ax = plt.subplot(grid[i])
# fig, axes = plt.subplots()
rcParams['figure.figsize'] = [10,5]
plt.style.use('fivethirtyeight') 
sns.set_style('whitegrid')

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 900)
pd.set_option('display.max_columns', 50)

# Setting specific package
from bs4 import BeautifulSoup
import requests
import re
from tqdm import tqdm

## Scraping Statistic Data
Source : teamrankings.com

<br> There are 115 team statistics from 1997 that we can get from teamrankings.com but here we will only scrape the data starting from 2010-2011 NCAA Regular Season. The statistics we get is the exact statistics recorded right before the NCAA Tournament start

In [0]:
# Define the base url for scraping
base_url = 'https://www.teamrankings.com'

In [0]:
# Get url for every statistics
main_url = base_url + '/ncb/team-stats/'
f = requests.get(main_url)
main_soup = BeautifulSoup(f.text)

# Save all the statistics url in list
list_stat_url = []
for url in main_soup.find_all(href=re.compile("/ncaa-basketball/stat/")) :
  list_stat_url.append(url['href'])

In [6]:
# Define supporting variable
# base_date is '-03-12' because from 2010-2011 season till 2020, the NCAA Tournment never start before that date
base_date = '-03-12'
list_year = [str(i) for i in range(2011,2020)]

# We take 5 different format for every statstics
base_var = ['Stat-', 'Last3-', 'Last1-', 'Home-', 'Away-']
dict_full = {}

# Iterate for every year
for year in tqdm(list_year) :

  # Make a dictionary to save all the dataframe each year
  dict_year = {}

  # Iterate for every statistics
  for stat in tqdm(list_stat_url) :
    
    # Define the url for the statistics
    url = base_url + stat + '?date=' + year + base_date

    # Take the HTML of the page
    f = requests.get(url)
    soup = BeautifulSoup(f.text)

    # Define varaible name
    var_stat = stat.split('/')[-1]
    list_var = [var+var_stat for var in base_var]
    dict_var = {var:[] for var in list_var}

    # Save the data into the dictionary
    stat_soup = soup.find_all('td', {"class": "text-right"})
    for i,val in enumerate(stat_soup) :
      
      # Save important value to the variable
      num = float(val['data-sort'])
      pointer = i % 6

      # Put it into the dictionary
      try :
        dict_var[list_var[pointer]].append(num)
      except :
        continue
      
    # Save the team name into the dictionary
    team_soup = soup.find_all('td', {"class": "text-left nowrap"})
    dict_var['Team'] = [team['data-sort'] for team in team_soup]

    # Make the dataframe and save it to dict_year
    df_stat = pd.DataFrame(dict_var)
    dict_year[stat] = df_stat

  # Save the dict_year into the full dict
  dict_full[year] = dict_year

100%|██████████| 9/9 [16:36<00:00, 112.40s/it]


In [7]:
# Merge all the dataframe from each year
list_df_year = []

for df in tqdm(dict_full.values()) :

  # Initiate a dataframe
  df_year = df[list(df.keys())[0]][['Team']]

  # Compile
  for little_df in df.values() :
    df_year = df_year.merge(little_df)

    # Buang variabel 'Team' yang jadi duplikat
    # df_year = df_year.loc[:,~df_year.columns.duplicated()]

  # Save it into list
  list_df_year.append(df_year)

100%|██████████| 9/9 [00:12<00:00,  1.33s/it]


In [8]:
# Compile all the dataframe into one final dataframe
df_full = pd.DataFrame()
year = 2011

for df in tqdm(list_df_year) :

  # Make variable 'Year'
  df['Year'] = year

  # Compile it
  df_full = pd.concat([df_full, df])

  # Update year value
  year = year + 1

100%|██████████| 9/9 [00:00<00:00, 121.69it/s]


In [0]:
# Save the dataset
df_full.to_csv('Data Teamranking/teamranking.csv', index=False)